In [3]:
!docker rm ollama

ollama


In [4]:
!docker run -d -v ollama:/root/.ollama -p 11434:11434 --name ollama ollama/ollama

58b44456c8e1efffdc1ec4ed0e326c7f1a790e279d96eb76ac8e2eafc2d4de8e


In [5]:
!docker ps

CONTAINER ID   IMAGE           COMMAND               CREATED         STATUS         PORTS                                           NAMES
58b44456c8e1   ollama/ollama   "/bin/ollama serve"   4 seconds ago   Up 2 seconds   0.0.0.0:11434->11434/tcp, :::11434->11434/tcp   ollama


In [18]:
#!docker exec -it ollama ollama run llama3.2

from tqdm import tqdm

from ollama import pull

current_digest, bars = '', {}
for progress in pull('llama3.2', stream=True):
  digest = progress.get('digest', '')
  if digest != current_digest and current_digest in bars:
    bars[current_digest].close()

  if not digest:
    print(progress.get('status'))
    continue

  if digest not in bars and (total := progress.get('total')):
    bars[digest] = tqdm(total=total, desc=f'pulling {digest[7:19]}', unit='B', unit_scale=True)

  if completed := progress.get('completed'):
    bars[digest].update(completed - bars[digest].n)

  current_digest = digest

pulling manifest


pulling dde5aa3fc5ff: 100%|██████████| 2.02G/2.02G [00:00<00:00, 4.60TB/s]
pulling 966de95ca8a6: 100%|██████████| 1.43k/1.43k [00:00<00:00, 12.5MB/s]
pulling fcc5a6bec9da: 100%|██████████| 7.71k/7.71k [00:00<00:00, 58.5MB/s]
pulling a70ff7e570d9: 100%|██████████| 6.02k/6.02k [00:00<00:00, 40.0MB/s]
pulling 56bb8bd477a5: 100%|██████████| 96.0/96.0 [00:00<00:00, 956kB/s]
pulling 34bb5ab01051: 100%|██████████| 561/561 [00:00<00:00, 5.03MB/s]

verifying sha256 digest
writing manifest
success


In [7]:
!pip install ollama

In [ ]:
from ollama import chat
from ollama import ChatResponse

response: ChatResponse = chat(model='llama3.2', messages=[
  {
    'role': 'user',
    'content': 'Why is the sky blue?',
  },
])
print(response['message']['content'])
# or access fields directly from the response object
#print(response.message.content)

The sky appears blue to our eyes due to a phenomenon called Rayleigh scattering. Here's what happens:

1. **Sunlight enters Earth's atmosphere**: When sunlight enters our atmosphere, it consists of a broad spectrum of wavelengths (colors).
2. **Wavelengths scatter differently**: The shorter wavelengths (like blue and violet) are scattered more than the longer wavelengths (like red and orange) by the tiny molecules of gases in the atmosphere, such as nitrogen and oxygen.
3. **Scattered light reaches our eyes**: As the sunlight passes through the atmosphere, the shorter wavelengths (blue and violet) are scattered in all directions and reach our eyes from all parts of the sky.
4. **Blue color prevails**: Since the shorter wavelengths are scattered more than the longer wavelengths, the blue light is distributed throughout the sky, making it appear blue to our eyes.

The other colors of the spectrum, like red and orange, are not scattered as much because they have a longer wavelength. That'

AttributeError: 'dict' object has no attribute 'message'

In [10]:
import ollama
ollama.list()

{'models': [{'name': 'llama3.2:latest',
   'model': 'llama3.2:latest',
   'modified_at': '2025-02-19T20:47:14.501209309Z',
   'size': 2019393189,
   'digest': 'a80c4f17acd55265feec403c7aef86be0c25983ab279d83f3bcd3abbcb5b8b72',
   'details': {'parent_model': '',
    'format': 'gguf',
    'family': 'llama',
    'families': ['llama'],
    'parameter_size': '3.2B',
    'quantization_level': 'Q4_K_M'}},
  {'name': 'llama2:latest',
   'model': 'llama2:latest',
   'modified_at': '2025-02-19T17:35:33.64834668Z',
   'size': 3826793677,
   'digest': '78e26419b4469263f75331927a00a0284ef6544c1975b826b15abdaef17bb962',
   'details': {'parent_model': '',
    'format': 'gguf',
    'family': 'llama',
    'families': ['llama'],
    'parameter_size': '7B',
    'quantization_level': 'Q4_0'}},
  {'name': 'mxbai-embed-large:latest',
   'model': 'mxbai-embed-large:latest',
   'modified_at': '2024-10-22T04:50:24.094364149Z',
   'size': 669615493,
   'digest': '468836162de7f81e041c43663fedbbba921dcea9b9fefea1

In [12]:
ollama.generate(model='llama3.2', prompt='Why is the sky blue?')['response']

"The sky appears blue because of a phenomenon called Rayleigh scattering, named after the British physicist Lord Rayleigh, who first described it in the late 19th century.\n\nHere's what happens:\n\n1. Sunlight enters Earth's atmosphere and consists of a spectrum of colors, including all the colors of the visible spectrum (red, orange, yellow, green, blue, indigo, and violet).\n2. When sunlight encounters tiny molecules of gases such as nitrogen (N2) and oxygen (O2) in the atmosphere, these molecules scatter the light in all directions.\n3. However, the shorter wavelengths of light, like blue and violet, are scattered more than the longer wavelengths, like red and orange. This is because the smaller molecules are more effective at scattering the shorter wavelengths.\n4. As a result, the blue light is dispersed throughout the atmosphere, giving the sky its blue appearance.\n5. The color of the sky can vary depending on factors such as:\n * Time of day: During sunrise and sunset, the sky

In [ ]:
from ollama import Client

def chat_with_ollama(model: str, prompt: str, questions: list, host: str):
    """
    Envia um prompt inicial para o modelo e, em seguida, processa uma lista de perguntas sequenciais.
    
    :param model: Nome do modelo a ser usado no Ollama.
    :param prompt: Texto inicial para contextualizar a conversa.
    :param questions: Lista de perguntas a serem enviadas ao modelo.
    :param host: Endereço do servidor Ollama.
    """
    client = Client(host=host)
    
    # Envia o prompt inicial
    response = client.chat(model=model, messages=[{"role": "user", "content": prompt}])
    print("Resposta inicial:", response["message"]["content"], "\n")
    
    # Processa cada pergunta na sequência
    for question in questions:
        response = client.chat(model=model, messages=[{"role": "user", "content": question}])
        print(f"Pergunta: {question}")
        print(f"Resposta: {response['message']['content']}\n")

modelo = "llama3.1"  # Substitua pelo modelo desejado
prompt_inicial = "Explique brevemente a teoria da relatividade."
lista_de_perguntas = [
    "O que é espaço-tempo?",
    "Como a gravidade afeta o tempo?",
    "Quais são as implicações práticas da teoria da relatividade?"
]
servidor = "http://177.93.135.244:8080/"

chat_with_ollama(modelo, prompt_inicial, lista_de_perguntas, servidor)


RemoteProtocolError: Server disconnected without sending a response.